In [1]:
import os
os.chdir('/kaggle/input/myfunctions')
from functions import *

In [2]:
!ls

functions.py


In [3]:
import tensorflow
from tensorflow.keras.layers import LSTM, Dense, Softmax, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2024-02-24 14:49:52.331460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 14:49:52.331567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 14:49:52.618757: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import pandas as pd
import numpy as np
import pickle
from functions import *
os.chdir('/kaggle/input/mydatasets2')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import xgboost as xg
from tqdm import tqdm

In [5]:
help(preprocess_data_equal_division)

Help on function preprocess_data_equal_division in module functions:

preprocess_data_equal_division(file_path, split=True, test_size=0.2, time_steps=10, num_stocks=30, le=False, only_close=False, equal_split=True, min_change=1e-05)



In [6]:
def process_data(args):
    # Process each item (example: perform some computation)
    df, k = args
    arr_list = []
    close_values = []
    df = df.reset_index()
    for i in range(0, len(df)-k):
      arr = df[['Open', 'High', 'Low', 'Volume']].iloc[i:i+k, :].values
      close_value = df['Close'].loc[i+k] - df['Close'].loc[i+k-1]
      arr_list.append(arr)
      close_values.append(close_value)
    return arr_list, close_values

In [7]:
import multiprocessing
def preprocess_data(df_list, split=True, time_steps = 10, num_stocks = 30, le=False, only_close = False, equal_split=True, min_change=1e-5, thread_name='1'):
  # num_stocks = 30 # The dataset is a list of 2000 stock dataframes. num_stocks is the number of stocks to consider for training (Matter of RAM capacity)
  # time_steps = 10 # Time steps to consider
      print(f"Thread {thread_name} started processing")
      k = time_steps
      num_cores = multiprocessing.cpu_count()
      arr_list = []
      close_values = []
      if not num_stocks:
        num_stocks = len(df_list)
      if not only_close:
          for df in df_list[:num_stocks]:
#             start_time = time.time() # start time
            chunk_size = len(df)//num_cores
            mp_df_list = []
            for i in range(num_cores):
                start_idx = i*chunk_size
                end_idx = start_idx + chunk_size if i < num_cores-1 else len(df)
                mp_df_list.append(df.iloc[start_idx:end_idx, :])
            with multiprocessing.Pool(processes=num_cores) as pool:
                results = pool.map(process_data, [(df_, k) for df_ in mp_df_list])
#             end_time = time.time() # end time
#             elapsed_time = end_time - start_time
#             print("Elapsed time:", elapsed_time, "seconds")
            for results1 in results:
                arr_list.extend(results1[0])
                close_values.extend(results1[1])

      else:
          for df in df_list[:num_stocks]:
            for i in range(0, len(df)-k-1):
              arr = df['Close'].loc[i:i+k].values
              close_value = df['Close'].loc[i+k+1] - df['Close'].loc[i+k]
              arr_list.append(arr)
              close_values.append(close_value)
      labels_list = []
      # Check if wants to have equal split of 'decrease', 'no big change', 
      if equal_split:
        combined_data = list(zip(close_values, arr_list))
        combined_data = sorted(combined_data, key=lambda x: x[0])
        k = len(combined_data)//3
        labels_list = ['decrease']*k + ['no big change']*k + ['increase']*(len(combined_data)-2*k)
        features = [data[1] for data in combined_data]
        features = np.array(features)
      else:
        for i in range(len(close_values)):
          if np.abs(close_values[i]) < min_change:
            labels_list.append('no big change')
          elif close_values[i] < 0:
            labels_list.append('increase')
          elif close_values[i] > 0:
            labels_list.append('decrease')
        features = np.array(arr_list)

      labels_array = np.array(labels_list)
      labels_encoded = np.zeros((len(labels_list), 3))
      labels_encoded[labels_array=='decrease', 0] = 1
      labels_encoded[labels_array=='no big change', 1] = 1
      labels_encoded[labels_array=='increase', 2] = 1

      if not le:
        labels = labels_encoded.copy()
      else:
        labels_array[labels_array=='decrease'] = 0
        labels_array[labels_array=='no big change'] = 1
        labels_array[labels_array=='increase'] = 2
        labels = labels_array.astype('int').copy()
      if split:
        x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, random_state=42)
        return x_train, x_valid, y_train, y_valid
      else:
        return features, labels
      print(f"Thread {thread_name} finished processing")


In [8]:
num_features = 4 # Number of features in the data (Example: Low, High, etc)
num_stocks = 300 # The dataset is a list of 2000 stock dataframes. num_stocks is the number of stocks to consider for training (Matter of RAM capacity)
num_epochs = 20
batch_size=64
units = 64
models_dict = {}
train_metrics_dict = {}
valid_metrics_dict = {}
test_metrics_dict = {}
time_steps = 20 # Time steps to consider
num_bagging_splits = 10
df_list = extract_df_list('/kaggle/input/mydatasets2/training_set1.pkl')
x_train, x_valid, y_train, y_valid = preprocess_data(df_list,split = True, time_steps=time_steps, num_stocks = num_stocks,
                                        only_close=False,
                                        equal_split=False)

Thread 1 started processing


In [9]:

x_train_end_ind = int(len(x_train)//num_bagging_splits)*num_bagging_splits
x_train_list = np.split(x_train[:x_train_end_ind, :], indices_or_sections=num_bagging_splits)
y_train_end_ind = int(len(y_train)//num_bagging_splits)*num_bagging_splits
y_train_list = np.split(y_train[:y_train_end_ind, :], indices_or_sections=num_bagging_splits)
for i in tqdm(range(num_bagging_splits)): 
    model = Sequential((LSTM(units, input_shape=(time_steps, 4), return_sequences=True),
                       LSTM(2*units, return_sequences=False),
                       Dense(256),
                       Dense(128),
                       Dense(3),
                       Softmax()))
    model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.001), metrics='Accuracy')
    callbacks = [EarlyStopping(patience=15, restore_best_weights=True, min_delta=1e-4), ReduceLROnPlateau(verbose=True)]
    model.fit(x_train_list[i], y_train_list[i], validation_data=(x_valid, y_valid), epochs=num_epochs, batch_size=batch_size)
    models_dict[i] = model

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/20


I0000 00:00:1708786388.115545    2205 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


796/796 [==============================] - 17s 15ms/step - loss: 0.7919 - Accuracy: 0.4862 - val_loss: 0.7868 - val_Accuracy: 0.4858
Epoch 2/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7888 - Accuracy: 0.4904 - val_loss: 0.7863 - val_Accuracy: 0.4954
Epoch 3/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7884 - Accuracy: 0.4913 - val_loss: 0.7885 - val_Accuracy: 0.4935
Epoch 4/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7884 - Accuracy: 0.4920 - val_loss: 0.7865 - val_Accuracy: 0.4923
Epoch 5/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7883 - Accuracy: 0.4916 - val_loss: 0.7859 - val_Accuracy: 0.4929
Epoch 6/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7875 - Accuracy: 0.4944 - val_loss: 0.7870 - val_Accuracy: 0.4938
Epoch 7/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7878 - Accuracy: 0.4942 - val_loss: 0.7877 - val_Accuracy: 0.4849
Epo

 10%|█         | 1/10 [03:42<33:24, 222.67s/it]

...

 80%|████████  | 8/10 [30:14<07:31, 225.74s/it]

Epoch 1/20
796/796 [==============================] - 16s 15ms/step - loss: 0.7920 - Accuracy: 0.4888 - val_loss: 0.7871 - val_Accuracy: 0.4935
Epoch 2/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7896 - Accuracy: 0.4940 - val_loss: 0.7861 - val_Accuracy: 0.4937
Epoch 3/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7889 - Accuracy: 0.4942 - val_loss: 0.7874 - val_Accuracy: 0.4946
Epoch 4/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7888 - Accuracy: 0.4936 - val_loss: 0.7888 - val_Accuracy: 0.4943
Epoch 5/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7885 - Accuracy: 0.4953 - val_loss: 0.7883 - val_Accuracy: 0.4935
Epoch 6/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7884 - Accuracy: 0.4991 - val_loss: 0.7860 - val_Accuracy: 0.4942
Epoch 7/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7882 - Accuracy: 0.4943 - val_loss: 0.7870 - val_Accuracy:

 90%|█████████ | 9/10 [33:55<03:44, 224.24s/it]

Epoch 1/20
796/796 [==============================] - 15s 15ms/step - loss: 0.7889 - Accuracy: 0.4888 - val_loss: 0.7862 - val_Accuracy: 0.4935
Epoch 2/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7862 - Accuracy: 0.4939 - val_loss: 0.7860 - val_Accuracy: 0.4936
Epoch 3/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7856 - Accuracy: 0.4879 - val_loss: 0.7898 - val_Accuracy: 0.4832
Epoch 4/20
796/796 [==============================] - 10s 13ms/step - loss: 0.7850 - Accuracy: 0.4894 - val_loss: 0.7863 - val_Accuracy: 0.4935
Epoch 5/20
796/796 [==============================] - 11s 14ms/step - loss: 0.7851 - Accuracy: 0.4912 - val_loss: 0.7869 - val_Accuracy: 0.4832
Epoch 6/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7849 - Accuracy: 0.4909 - val_loss: 0.7880 - val_Accuracy: 0.4935
Epoch 7/20
796/796 [==============================] - 11s 13ms/step - loss: 0.7849 - Accuracy: 0.4884 - val_loss: 0.7862 - val_Accuracy:

100%|██████████| 10/10 [37:35<00:00, 225.58s/it]


In [10]:
x_test, y_test = preprocess_data_equal_division('training_set11.pkl', 
                                                    split=False, 
                                                    time_steps=time_steps, 
                                                    num_stocks = num_stocks,
                                                    only_close=False,
                                                    equal_split=False)

In [ ]:
for i, model in models_dict.items():
    print(f"----------Model - {i}---------")
    test_metrics_dict[i] = metric_calculations_categorical(model, x_test, y_test, set_="Test set")
    train_metrics_dict[i] = metric_calculations_categorical(model, x_train, y_train, set_="Training set")
    valid_metrics_dict[i] = metric_calculations_categorical(model, x_valid, y_valid, set_="Validation set")

In [ ]:
lis = [models_dict,
train_metrics_dict, 
valid_metrics_dict, 
test_metrics_dict]

In [ ]:
file_path = '/kaggle/working/lstm_olhv_100.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(lis, file)

### Bagging

In [17]:
def get_bagging_metrics(models, x, y_true, batch_size=64, set_=""):
    y_pred = np.zeros_like(y_true)
    for model in models:
        y_pred += model.predict(x, batch_size=512, verbose=False)
    y_pred = y_pred/len(models)
    y_pred_args = np.argmax(y_pred, axis=1)
    y_true_args = np.argmax(y_true, axis=1)
    return metric_calculations(y_pred_args, y_true_args, set_ = set_)

In [19]:
models_list = list(models_dict.values())
test_metrics_dict[i] = get_bagging_metrics(models_list, x_test, y_test, set_="Test set")
train_metrics_dict[i] = get_bagging_metrics(models_list, x_train, y_train, set_="Training set")
valid_metrics_dict[i] = get_bagging_metrics(models_list, x_valid, y_valid, set_="Validation set")

For Test set
  Accuracy: 48.792481779823554,
  Precision: 48.67178934246862,
  Recall: 83.80451045363037,
  Specificity: 15.196819025713879,
  F1score: 61.57955022937496
For Training set
  Accuracy: 49.534833490417846,
  Precision: 49.425328323912716,
  Recall: 93.72472935311119,
  Specificity: 6.412340496521864,
  F1score: 64.72055401890198
For Validation set
  Accuracy: 49.55623625510525,
  Precision: 49.45216159346785,
  Recall: 93.75695349502527,
  Specificity: 6.374025651377287,
  F1score: 64.75124173321258
